In [4]:
from ase import Atoms
from mace.calculators import mace_mp
from ase.build import surface
from ase.constraints import FixAtoms, UnitCellFilter
from ase.optimize import QuasiNewton
from ase.visualize import view
from ase.io import read, write
from ase.optimize import LBFGS
import numpy as np
import pandas as pd
from tabulate import tabulate
from cu2o_bulk import cu2o_bulk, cu2o111, CuDO_FCC111

bulk = cu2o_bulk()
bulk.calc = mace_mp(model="large", dispersion=True, default_dtype="float64", device='cpu')
ucf = UnitCellFilter(bulk)
LBFGS(ucf).run(fmax=0.01)
E_bulk = bulk.get_potential_energy()
bulk_oxygens = bulk[bulk.symbols=='O']
print(f"Number of Oxygen atoms in bulk: {len(bulk_oxygens)}")
#write('bulk.xyz', bulk)

n_layers = { 3,4,5,6,7,8,9,10}
Cu2Otable={}

n_layers_list=[]
slab_oxygens_list=[]
E_surf_list=[]


for n_layers in range(3,11):
  vacuum=10
  slab = CuDO_FCC111(bulk, n_layers, vacuum)
  #write('slab.xyz', slab)
  slab_oxygens = slab[slab.symbols=='O']
  Cu2Otable[n_layers]=len(slab_oxygens)
  print(f"Number of Oxygen atoms in slab: {len(slab_oxygens)}")

  slab.calc = mace_mp(model="large", dispersion=True, default_dtype="float64", device='cpu')
  E_slab = slab.get_potential_energy()
  E_cleav = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])
  print(f'{n_layers=} {E_cleav=}')

  qn = LBFGS(slab, trajectory='CuD_Os.traj')
  qn.run(fmax=0.01)
  E_slab = slab.get_potential_energy()

  ##Calc Surface Energy
  E_surf = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])
  print(f'{n_layers=} {E_surf=}')

  n_layers_list.append(n_layers)
  slab_oxygens_list.append(len(slab_oxygens))
  E_surf_list.append(E_surf)

df = pd.DataFrame({'Number of Layers': n_layers_list, 'Number of Oxygen Atoms':slab_oxygens_list, 'Surface Energy': E_surf_list})
table=tabulate(df, headers = 'keys', tablefmt = 'fancy_grid')
df.to_csv('CuD_Os.csv', index=False)
print(table)


'\nE_slab = slab.get_potential_energy()\nE_cleav = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])\nprint(f\'{n_layers=} {E_cleav=}\')\n\nslab = CuDO_FCC111(n_layers=5, vacuum=10)\n#slab.calc = mace_mp(model="medium", dispersion=False, default_dtype="float64", device=\'cpu\')\n\nbottom_Cu_z = np.min(slab[slab.symbols==\'Cu\'].positions[:,2])\nmask1=slab.positions[:, 2] < bottom_Cu_z + 1.0\nslab.set_constraint(FixAtoms(mask=mask1))\n\n#qn = LBFGS(slab, trajectory=\'111slab.traj\')\n#qn.run(fmax=0.01)\n#E_slab = slab.get_potential_energy()\nt = read(\'111slab.traj@:\')\natoms = t[-1]\nview(atoms)\n  \nE_surf = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])\nprint(f\'{n_layers=} {E_surf=}\')\n'